In [78]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OrdinalEncoder 
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler 
from sklearn.preprocessing import StandardScaler

In [3]:
data = pd.read_csv('travel_insurance.csv')

In [4]:
data.head(5)

,Agency,Agency Type,Distribution Channel,Product Name,Claim,Duration,Destination,Net Sales,Commission (in value),Gender,Age
0,JZI,Airlines,Online,Value Plan,0,12,"TAIWAN, PROVINCE OF CHINA",45.0,15.75,M,39
1,EPX,Travel Agency,Online,Cancellation Plan,0,50,JAPAN,22.0,0.00,None,36
2,EPX,Travel Agency,Online,Cancellation Plan,0,251,BRAZIL,80.0,0.00,None,36
3,EPX,Travel Agency,Online,Cancellation Plan,0,6,INDIA,-10.0,0.00,None,36
4,JZI,Airlines,Online,Value Plan,0,5,CHINA,45.0,15.75,M,34


Признаки, с которыми будем работать:


Agency — название страхового агентства

Agency Type — тип страхового агентства

Distribution Channel — канал продвижения страхового агентства

Product Name — название страхового продукта

Duration — длительность поездки (количество дней)

Destination — направление поездки

Net Sales — сумма продаж ($)

Commission — комиссия страхового агентства ($)

Gender — пол застрахованного

Age — возраст застрахованного

Целевой признак:
Claim (англ. «претензия») — потребовалась ли страховая выплата: «да» — 1, «нет» — 0

In [5]:
data.describe()

,Claim,Duration,Net Sales,Commission (in value),Age
count,50660.000000,50660.000000,50660.000000,50660.000000,50660.000000
mean,0.014864,49.587150,40.749961,9.800755,39.957284
std,0.121009,103.629611,48.470030,19.716468,14.027618
min,0.000000,-2.000000,-389.000000,0.000000,0.000000
25%,0.000000,9.000000,18.000000,0.000000,35.000000
50%,0.000000,22.000000,27.000000,0.000000,36.000000
75%,0.000000,53.000000,49.000000,11.550000,43.000000
max,1.000000,4881.000000,682.000000,262.760000,118.000000



Разбейте исходные данные на две выборки:

обучающую (train);
валидационную (valid). Это 25% исходных данных.

Установите параметр (random_state) равным 12345.

Объявите четыре переменные и запишите в них:

признаки: features_train, features_valid;

целевой признак: target_train, target_valid.

Выведите на экран размеры таблиц, которые хранятся в переменных: features_train и features_valid.

In [16]:
features = data.drop(['Claim'],axis=1)

In [18]:
target = data['Claim']

In [21]:
features_train, features_valid,target_train, target_valid = train_test_split(features,target,test_size=0.25,random_state=12345)

In [23]:
print(features_train.shape)
print(features_valid.shape)

(37995, 10)
(12665, 10)


Принадлежность к категории логистическая регрессия вычисляет по формуле, состоящей из признаков. Они могут быть только численные. Наши данные содержали и категориальные признаки тоже — в этом и была ошибка.

Проверьте, признаки какого типа хранятся в таблице. Выведите их на экран. Затем напечатайте первые пять значений столбца Gender.

In [28]:
print(data.dtypes)

Agency                    object
Agency Type               object
Distribution Channel      object
Product Name              object
Claim                      int64
Duration                   int64
Destination               object
Net Sales                float64
Commission (in value)    float64
Gender                    object
Age                        int64
dtype: object


In [29]:
print(data['Gender'].head(5))

0       M
1    None
2    None
3    None
4       M
Name: Gender, dtype: object


Прямое кодирование
Преобразовать категориальные признаки в численные поможет техника прямого кодирования, или отображения (англ. One-Hot Encoding, OHE).
Принцип работы One-Hot Encoding объясним на значениях признака Gender.
Для каждого значения признака Gender (F, M, None) создаётся столбец: 
Gender_F (от англ. female, «женщина»),
Gender_M (от англ. male, «мужчина»),
Gender_None (данных о поле нет).
Когда ставить единицу в добавленные столбцы, зависит от того, каково значение Gender. Например, если F, то 1 записывается в столбец Gender_F, а M — в Gender_M. 
image

 
Резюмируем. Техникой OHE категориальные признаки переводятся в численные в два этапа:
Для каждого значения признака создаётся новый столбец;
Если объекту категория подходит, присваивается 1, если нет — 0.
Новые признаки (Gender_F, Gender_M, Gender_None) называются дамми-переменными, или дамми-признаками (англ. dummy variable, «фиктивная переменная»).
Для прямого кодирования в библиотеке pandas есть функция pd.get_dummies() (от англ. «получить фиктивные переменные»).

## Преобразуйте колонку Gender техникой OHE.
Вызовите функцию pd.get_dummies() и напечатайте на экране первые пять записей изменённой таблицы.

In [32]:
print(pd.get_dummies(data['Gender'].head(5)))

   M  None
0  1     0
1  0     1
2  0     1
3  0     1
4  1     0


Когда данных в избытке, можно угодить в ловушку фиктивных признаков.

### Дамми-ловушка

Пример:
тобы подать документы на шенгенскую визу, нужно доказать, что деньги у вас есть.

Вы решили перестраховаться, поэтому взяли и выписку с банковского счёта, и справку с работы, и 2-НДФЛ. Хотя визовому центру достаточно двух документов. Вашей модели лишняя информация тоже не очень-то нужна. 

Если оставить всё как есть, обучаться она будет сложнее. 
В таблицу добавились три новых столбца. Поскольку они сильно связаны между собой, один удалим без сожаления. Восстановить столбец можно по оставшимся двум. Так мы не попадём в дамми-ловушку (англ. dummy trap, «ловушка фиктивных признаков»).


Столбец уберём вызовом функции:

### pd.get_dummies() с аргументом drop_first (от англ. «сбросьте первый»).

Он удаляет первую колонку и передаётся как drop_first=True или drop_first=False (True — первый столбец сбрасывается, False — не сбрасывается).

In [35]:
print(pd.get_dummies(data['Gender'],drop_first=True).head(5))

   M  None
0  1     0
1  0     1
2  0     1
3  0     1
4  1     0



Примените прямое кодирование ко всему датафрейму. Вызовите функцию pd.get_dummies() c аргументом drop_first. Сохраните таблицу в переменной data_ohe.
Выведите на экран первые три строки преобразованной таблицы.

In [37]:
data_ohe = pd.get_dummies(data,drop_first=True)

In [39]:
data_ohe.head(3)

,Claim,Duration,Net Sales,Commission (in value),Age,Agency_ART,Agency_C2B,Agency_CBH,Agency_CCR,Agency_CSR,...,Destination_UNITED STATES,Destination_URUGUAY,Destination_UZBEKISTAN,Destination_VANUATU,Destination_VENEZUELA,Destination_VIET NAM,Destination_ZAMBIA,Destination_ZIMBABWE,Gender_M,Gender_None
0,0,12,45.0,15.75,39,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0,50,22.0,0.00,36,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,251,80.0,0.00,36,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


Разбейте исходные данные на две выборки в соотношении 75:25 (%):
обучающую (train);
валидационную (valid).

Объявите четыре переменные и запишите в них:
признаки: features_train, features_valid;
целевой признак: target_train, target_valid.

Вам предстоит освоить альтернативный способ работы с функцией train_test_split(): когда на вход подаются две переменные (признаки и целевой признак). Поработайте с документацией.

Обучите логистическую регрессию.
Напечатайте на экране текст "Обучено!" (уже в прекоде). Так вы убедитесь, что код выполнился без ошибок.

Вложите и в train_test_split(), и в LogisticRegression() параметр random_state, равный 12345.

In [57]:
data_ohe = pd.get_dummies(data, drop_first=True)
target = data_ohe['Claim']
features = data_ohe.drop('Claim', axis=1)

In [60]:
features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size = 0.25, random_state = 12345)

In [61]:
model = LogisticRegression(random_state=12345,solver='liblinear')
model.fit(features_train, target_train)
print("Обучено!")

Обучено!


### Порядковое кодирование

Расскажем о другой технике кодирования признаков в решающем дереве и случайном лесе.

Ordinal Encoding (от англ. «кодирование по номеру категории»). Она работает так:

Фиксируется, какой цифрой кодируется класс;

Цифры размещаются в столбце.

Техника подходит для преобразования признаков в решающем дереве и случайном лесе (он состоит из деревьев).

Чтобы выполнить кодирование, в sklearn есть структура данных OrdinalEncoder (англ. «порядковый кодировщик»). Она находится в модуле sklearn.preprocessing (от англ. «предобработка»). 
Импортируем OrdinalEncoder из библиотеки:

from sklearn.preprocessing import OrdinalEncoder 
Преобразование выполняется в три этапа: 

Создаём объект этой структуры данных.
encoder = OrdinalEncoder() 

Чтобы получить список категориальных признаков, вызываем метод fit() — как и в обучении модели. Передаём ему данные как аргумент.
encoder.fit(data) 

Преобразуем данные функцией transform() (англ. «преобразовать»). Изменённые данные будут храниться в переменной data_ordinal (англ. «порядковые данные»).
data_ordinal = encoder.transform(data) 

Чтобы код добавил названия столбцов, оформим данные в структуру DataFrame():
### data_ordinal = pd.DataFrame(encoder.transform(data), columns=data.columns) 
                                                      
Если преобразование признаков требуется лишь один раз, как в нашей задаче, код можно упростить вызовом функции fit_transform() (от англ. «подогнать и преобразовать»).
Она объединяет функции: fit() и transform(). 


### data_ordinal = pd.DataFrame(encoder.fit_transform(data), columns=data.columns) 

Преобразуйте признаки техникой Ordinal Encoding.
Импортируйте OrdinalEncoder из модуля sklearn.preprocessing. 

Сохраните результат в переменной data_ordinal. Оформите данные в структуру DataFrame().

Напечатайте на экране первые пять строк таблицыю

In [63]:
encoder = OrdinalEncoder()

In [66]:
data_ordinal = pd.DataFrame(encoder.fit_transform(data), columns = data.columns)

In [67]:
print(data_ordinal.head(5))

   Agency  Agency Type  Distribution Channel  Product Name  Claim  Duration  \
0     9.0          0.0                   1.0          24.0    0.0      14.0   
1     7.0          1.0                   1.0          10.0    0.0      52.0   
2     7.0          1.0                   1.0          10.0    0.0     253.0   
3     7.0          1.0                   1.0          10.0    0.0       8.0   
4     9.0          0.0                   1.0          24.0    0.0       7.0   

   Destination  Net Sales  Commission (in value)  Gender   Age  
0        121.0      602.0                  486.0     1.0  37.0  
1         61.0      470.0                    0.0     2.0  34.0  
2         16.0      735.0                    0.0     2.0  34.0  
3         54.0      121.0                    0.0     2.0  34.0  
4         24.0      602.0                  486.0     1.0  32.0  



Обучите решающее дерево на преобразованных данных. Напечатайте на экране текст  "Обучено!"  (уже в прекоде). Так вы убедитесь, что код выполнился без ошибок

In [69]:
features = data_ordinal.drop('Claim',axis=1)
target = data_ordinal['Claim']

In [70]:
features_train, features_valid, target_train, target_valid = train_test_split(features,target,test_size=0.25,random_state=12345)

In [74]:
model = DecisionTreeClassifier(random_state=12345)

In [75]:
model.fit(features_train,target_train)

DecisionTreeClassifier(random_state=12345)

In [76]:
print('Обучено!')

Обучено!


### Подведём итог:

Когда все признаки должны стать количественными-подходит техника OHE;

Если все признаки категориальные, и их нужно преобразовать в числа — Ordinal Encoding.

Почему Ordinal Encoding не подходит для логистической регрессии? Она всё норовит посчитать по формуле. Если речь идёт о признаке Age, то это разумно, а вот с Gender есть трудности. Например, сложив значения «1» и «0» («женщина» и «мужчина») и разделив на «2», «средний пол» не получить.

### Масштабирование признаков

Что делать, если у признаков разный масштаб? Стандартизировать их!
В данных есть столбцы: Age и Commission. Допустим, возможен возраст от 0 до 100 лет, а страховая комиссия — от 100 долларов до 1000. Значения и их разбросы в столбце Commission больше, поэтому алгоритм автоматически решит, что этот признак важнее возраста. А это не так: все признаки значимы.
Чтобы избежать этой ловушки, признаки масштабируют — приводят к одному масштабу.
Один из методов масштабирования — стандартизация данных. 
Предположим, что все признаки распределены нормально, среднее (англ. mean, M) и дисперсия (лат. dispersio, D) определяются по выборке. Значения признака преобразуются по формуле:
Новое значение = Старое значение - М, поделённое на квадратный корень из D.
У нового признака устанавливается среднее, равное 0, и дисперсия, равная 1.
В sklearn есть отдельная структура для стандартизации данных — StandardScaler (от англ. «преобразователь масштаба методом стандартизации»). Он находится в модуле sklearn.preprocessing. 

Стандартизируйте численные признаки. Импортируйте StandardScaler из модуля sklearn.preprocessing.
Создайте объект структуры StandardScaler() и настройте его на обучающих данных. В переменной numeric уже есть список всех численных признаков.
Сохраните преобразованные обучающую и валидационную выборки в переменных: features_train и features_valid. 
Напечатайте на экране первые пять строк таблицы.

При записи изменённых признаков в исходный датафрейм код может вызывать предупреждение SettingWithCopy. Причина в особенности поведения sklearn и pandas.  Специалисты уже привыкли игнорировать такое сообщение.

In [88]:
#Чтобы предупреждение не появлялось, в код добавляют строчку:
pd.options.mode.chained_assignment = None
scaller = StandardScaler()
scaller.fit(features_train[numeric])

StandardScaler()

In [89]:
numeric = ['Duration', 'Net Sales', 'Commission (in value)', 'Age']

In [90]:
features_train[numeric] = scaller.transform(features_train[numeric])
features_valid[numeric] = scaller.transform(features_valid[numeric])

In [91]:
print(features_train.head(5))

       Agency  Agency Type  Distribution Channel  Product Name  Duration  \
33312     7.0          1.0                   1.0          10.0  0.422711   
50154     7.0          1.0                   1.0          10.0 -0.392566   
26729     7.0          1.0                   1.0          10.0 -0.492687   
37842     7.0          1.0                   1.0           1.0 -0.077897   
23588     7.0          1.0                   1.0           1.0 -0.564203   

       Destination  Net Sales  Commission (in value)  Gender       Age  
33312         24.0  -0.638946              -0.770269     2.0 -0.298857  
50154        113.0  -0.732311              -0.770269     2.0 -0.298857  
26729         76.0  -1.053254              -0.770269     2.0 -0.807193  
37842        124.0   0.428917              -0.770269     2.0 -0.383579  
23588         76.0  -0.388027              -0.770269     2.0 -0.468302  
